# 데이터 전처리

마운트 폴더 지정

In [38]:
import sys

In [39]:
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/src')

데이터 불러오기

In [40]:
import helper_functions
import pet.modeling
import pet.wrapper

In [41]:
import pandas as pd
import numpy as np

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pet
import os
import numpy as np
import tqdm
import datasets
import pandas as pd

yelp_review_full(2중)

In [43]:
def get_yelp_polarity_data(show_progress_bars=False):

    if not show_progress_bars: datasets.logging.disable_progress_bar()
    yelp_polarity_dataset = datasets.load_dataset('yelp_polarity')
    df_train = pd.DataFrame(yelp_polarity_dataset['train'])
    df_test = pd.DataFrame(yelp_polarity_dataset['test'])
    df_train['label'] = (df_train['label'] + 1).astype(str)
    df_test['label'] = (df_test['label'] + 1).astype(str)
    return df_train, df_test

In [44]:
df_origin, df_test = get_yelp_polarity_data(show_progress_bars=False)

2023-08-04 14:17:42,383 - WARNING - builder - Found cached dataset yelp_polarity (C:/Users/user/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


전처리 (대문자 소문자 통일, 중복 특수문자 제거)

In [45]:
import re

In [46]:
def cleantext(text):
    text = re.sub(r"[^A-Za-z0-9G-H,.\?!\"']", " ", text)
    text = re.sub(r"([,.\?!\"'])\1+", r"\1", text)
    text = text.replace("n n", '')
    text = re.sub(r" +", ' ', text)
    return text

In [47]:
df_origin['text'] = df_origin['text'].apply(lambda x : cleantext(x))

문장 길이 제한

In [52]:
df_origin_train = df_origin[df_origin['text'].str.len().between(300, 400)].reset_index(drop=True)

In [53]:
df_origin_train

,text,label
0,Wing sauce is like water. Pretty much a lot of...,1
1,I drove by yesterday to get a sneak peak. It r...,2
2,"Used to go there for tires, brakes, etc. Their...",1
3,I've been informed by a fellow Yelper that the...,2
4,Definitely a greasy spoon! Always packed here ...,2
...,...,...
62013,Hard choosing a place to eat when you're on th...,2
62014,Love this place! The portions are generous but...,2
62015,Having moved 15 times and having been referred...,1
62016,We had the best experience. The movers were su...,2


train data(문장 제한한 데이터에서 라벨 비율 맞추기)

In [54]:
def sample_data_by_label(dataframe, label_column, label_values, num_samples, random_state=42):
    sampled_data = pd.concat([dataframe[dataframe[label_column] == label].sample(n=num_samples, random_state=random_state)
                             for label in label_values], ignore_index=True)
    return sampled_data

In [55]:
df_train = sample_data_by_label(df_origin_train, 'label', ['1', '2'], 50, random_state=42)

In [56]:
df_train.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_train.csv", index = False)

In [58]:
# unlabeled data  = df_origin_train - df_train
def merge_and_filter(dataframe1, dataframe2, on_columns, how='left', num_samples=1000):

    merged_df = dataframe1.merge(dataframe2, on=on_columns, how=how, indicator=True)


    filtered_df = merged_df[merged_df['_merge'] == 'left_only']


    filtered_df = filtered_df.drop(columns=['_merge']).reset_index(drop=True)


    num_samples = min(num_samples, filtered_df.shape[0])
    random_indices = np.random.choice(filtered_df.index, num_samples, replace=False)
    filtered_df = filtered_df.loc[random_indices].reset_index(drop=True)

    return filtered_df

In [59]:
# random한 값으로 20100개 뽑기(unlabeled data  = df_train - df_train_400_train)
df_rest = merge_and_filter(df_origin_train, df_train, ['label', 'text'], how='left', num_samples=20100)

In [65]:
df_eval = df_rest[:100]
df_unlabeled = df_rest[100:10100]
df_test = df_rest[10100:]

In [66]:
df_unlabeled.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_unlabeled.csv", index = False)
df_eval.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_eval.csv", index = False)
df_test.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data3/df_test.csv", index = False)

test data

In [25]:
df_test.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET - 복사본/data/df_test.csv", index = False)